In [ ]:
import sys
sys.path.append('../')

In [ ]:
from api.utils.transformer import Transformer
from api.tests.test_parser import get_training_data

In [ ]:
training_data = get_training_data(9999999999)

In [ ]:
transformer = Transformer()
X, y = transformer.fit_transform(training_data)

In [ ]:
X_train = X[X.index < 13500]
X_test = X[X.index >= 13500]
y_train = y.loc[X_train.index]
y_test = y.loc[X_test.index]

# Build a model

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
et = ExtraTreesRegressor(n_jobs=-1, n_estimators=500)
et.fit(X_train, y_train)

# Evaluate model

In [ ]:
def get_scores(x, y, group, threshold=0.50):
    corr = spearmanr(x, y)[0]
    mu_a = x.groupby(group).mean()
    mu_p = y.groupby(group).mean()
    mu_a_top = mu_a[mu_a.rank(ascending=False) / len(mu_a) < threshold]
    mu_p_top = mu_p[mu_p.rank(ascending=False) / len(mu_p) < threshold]
    overlap = mu_p_top.index.isin(mu_a_top.index).mean()
    return pd.Series([corr, overlap])

# Save your model

In [ ]:
from api.utils.model import TWCModel

In [ ]:
model = TWCModel()
model.transformer = transformer
model.model = et
model.save('etmodel.p')